In [9]:
# Import necessary packages
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from urllib.request import urlopen
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [10]:
list_of_genres = ['children', 'fantasy', 'magical', 'mystery', 'action', 'crime', 'drama', 'science fiction', 'adventure', 'biography', 'comedy', 'thriller', 'musical', 'western', 'romance', 'animation', 'war', 'horror', 'sport', 'political', 'saga', ' satire', 'social']
#war_grades = {'children':1, 'fantasy':3, 'magical':3, 'mystery':4, 'action':7, 'crime':8, 'drama':7, 'SF':5, 'adventure': 4, 'biography':5, 'comedy':4, 'thriller':4, 'musical':4, 'western':8, 'romance': 2, 'war':10, 'horror':4, 'sport':3, 'political':6, 'saga':3, 'satire':2, 'social':4, 'animation':2}
#war_grades_df = pd.DataFrame(war_grades.items(), columns=['Genres', 'Grade'])
#war_grades_df.set_index('Genres')

In [11]:
#Function that will scrape a wikipedia page with a list of movies by year, add the year as a new column and return 
# a data frame.

def get_table_after_1996(year):
    #create the url 
    url = 'http://en.wikipedia.org/wiki/List_of_American_films_of_'+ str(year)
    html = urlopen(url) 
    soup = BeautifulSoup(html, 'html.parser')
    website_url = requests.get(url).text 
    soup = BeautifulSoup(website_url, 'lxml')
    My_table = soup.find('table',{'class':'wikitable sortable'})
    titles = []
    directors = []
    actors = []
    genres = []
    rows = My_table.find_all('tr')
    for row in rows:
        cells = row.find_all('td')
        if len(cells)>=1:
            title = cells[0]
            titles.append(title.text)
            director = cells[1]
            directors.append(director.text)
            actor = cells[2]
            actors.append(actor.text)
            genre = cells[3]
            genres.append(genre.text)
            index = [*range(1,len(titles)+1,1)]
            df = pd.DataFrame({'Titles': titles,'Directors' : directors, 'Actors': actors, 'Genres': genres, }, index = index   )
# add a year column             
            #years = [year]*len(df)
            #df['Year'] = years
#create a list with all the genres from the "Genres" column and lower the case of each element
            list_year_1 = df['Genres'].to_list()
            lower_list = [x.lower() for x in list_year_1]
#separate the genres where a movie has two or more genres
            list_final = [x.split() for x in lower_list]
            list_year = []
            for sublist in list_final:
                for item in sublist:
                    list_year.append(item)  
#some genres have comma attached, take off comma
            list_year_no_comma = []
            for item in list_year:
                if ',' in item:
                        list_year_no_comma.append(item[:-1])
                else:
                            list_year_no_comma.append(item)
#print(list_year_no_comma)
# Create a dictionary key- genres, value = number of movies of a certain genre
            genre_counts = {}
# Iterate through genre list
            for genre in list_year_no_comma:
# Update word counts - keep only genres in the general list of genres
                if genre in list_of_genres:
                    genre_counts[genre] = genre_counts.get(genre, 0) + 1 #Get previous entry, update by 1
                else:
                    continue
#Set 'Genres' as df index
            df_to_plot = pd.DataFrame(genre_counts.items(), columns=['Genres', 'year_1999'])
            df_to_plot.set_index('Genres')
# Join the the 2 Dataframes on 'Genres' column and replace NANs with mean value.
            #Zeitgeist_df1 = war_grades_df.join(df_to_plot.set_index('Genres'), on='Genres')
            #Zeitgeist_df1 = Zeitgeist_df1.fillna(5)
#Create a column year_war_grade 
            #Zeitgeist_df1['year_war_grade'] = Zeitgeist_df1['Grade']*Zeitgeist_df1['year_1999']
            #Zeitgeist_df1
# Add a year column
            #year = 1999
            years_final = [year]*len(df_to_plot)
            df_to_plot['Year'] = years_final
           
    return df_to_plot


In [16]:
get_table_after_1996(2018)

,Genres,year_1999,Year


In [3]:
pieces = []
for year in [*range(1999, 2004,1)]:
    url = 'http://en.wikipedia.org/wiki/List_of_American_films_of_'+ str(year)
    df = get_table_after_1996(year)
    pieces.append(df)



NameError: name 'urlopen' is not defined

In [5]:
df_all = pd.concat(pieces)
df_all
#df_all.to_csv('9_11.csv', index=False, header= True)

In [6]:
#grouped = df_all.groupby(['Year'])['year_war_grade'].sum()
#grouped.to_csv('grouped.csv', index='Year', header=True)

In [7]:
#grouped



Year
1999    1375.0
2000    1279.0
2001    1364.0
2002    1495.0
2003    1547.0
Name: year_war_grade, dtype: float64